In [28]:
import os
import subprocess
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# ===============================
# 配置 API Key（请替换为实际 Key）
# ===============================
os.environ["ZHIPUAI_API_KEY"] = "d9ae4d81e22cb9483f5b4d875ba2d1c1.0QNkBJCwDX7rLdI9"

# ===============================
# 下面为问答部分采用 langgraph 调用 LLM
# ===============================
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langgraph.checkpoint.memory import MemorySaver

# 定义问答状态结构
class QAState(TypedDict):
    messages: Annotated[list, add_messages]

# 构建 langgraph 状态图（用于单次问答调用）
qa_graph_builder = StateGraph(QAState)

# 初始化 ChatZhipuAI 模型（参数根据实际情况调整）
chat = ChatZhipuAI(
    model="glm-4-plus",
    temperature=0.5,
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# 定义问答节点函数，调用模型进行问答
def qa_node(state: QAState) -> QAState:
    response = chat.invoke(state["messages"])
    return {"messages": [response]}

qa_graph_builder.add_node("qa", qa_node)
qa_graph_builder.add_edge(START, "qa")
qa_graph_builder.add_edge("qa", END)
memory = MemorySaver()
qa_app = qa_graph_builder.compile(checkpointer=memory)

def qa_invoke(messages: list) -> object:
    """
    使用 langgraph 调用 LLM 进行问答。
    输入：对话历史 messages（列表，每个元素为字典，格式为 {"role": ..., "content": ...}）
    返回：模型输出（通常为最后一次问答响应）
    """
    input_state = {"messages": messages}
    output_state = qa_app.invoke(input_state)
    # 取返回的最后一条消息作为本次响应
    return output_state["messages"][-1]

# ===============================
# 以下为流水线其他部分（均为常规 Python 代码）
# ===============================

def analyze_pcap(pcap_file: str) -> list:
    """
    调用 shell 脚本运行 zeek 分析指定 pcap 文件，生成多个 .log 文件。
    返回生成的 .log 文件列表。
    """
    print("【Step 1】分析 pcap 文件...")
    # TODO: 替换为实际调用命令，如：
    # subprocess.run(["sh", "analyze_pcap.sh", pcap_file], check=True)
    # 这里模拟返回结果：
    log_files = ["capture1.log", "capture2.log", "capture3.log"]
    print(f"生成的 log 文件：{log_files}")
    return log_files

def convert_logs(log_files: list, n: int, m: int) -> list:
    """
    调用 shell 脚本，将 .log 文件转换为 .txt 文件，并删除每个文件的前 n 行与最后 m 行。
    返回转换得到的 .txt 文件列表。
    """
    print("【Step 2】转换 .log 文件为 .txt 文件...")
    txt_files = []
    for log in log_files:
        # TODO: 替换为实际调用，如：
        # subprocess.run(["sh", "convert_logs.sh", log, str(n), str(m)], check=True)
        # 模拟转换结果：例如只有文件名包含 "capture1" 和 "capture3" 的文件为有效
        if "capture1" in log or "capture3" in log:
            txt_files.append(log.replace(".log", "_useful.txt"))
        else:
            txt_files.append(log.replace(".log", "_useless.txt"))
    print(f"转换得到的 txt 文件：{txt_files}")
    return txt_files

def filter_effective_txt(txt_files: list) -> list:
    """
    筛选出有效的 .txt 文件（例如文件名中包含 "useful"）。
    返回有效文件列表。
    """
    effective = [f for f in txt_files if "useful" in f]
    print(f"筛选出有效 txt 文件：{effective}")
    return effective

def process_txt_file(txt_file: str) -> list:
    """
    处理 txt 文件，提取特定字段后生成新的 y1 文件。
    由于一个 txt 文件可能对应多个 y1 文件（即多个任务），
    返回 y1 文件路径列表。
    """
    print(f"【Step 3】处理 {txt_file}，提取字段生成 y1 文件...")
    # TODO: 实现字段提取逻辑，此处模拟：
    # 假设如果文件名中含 "capture1" 则生成 2 个任务，否则生成 1 个任务
    if "capture1" in txt_file:
        y1_files = [txt_file.replace("_useful.txt", f"_y1_{i}.txt") for i in [1, 2]]
    else:
        y1_files = [txt_file.replace("_useful.txt", "_y1.txt")]
    print(f"生成的 y1 文件：{y1_files}")
    return y1_files

def split_into_chunks(file_path: str, lines_per_chunk: int = 30) -> list:
    """
    将指定文件内容按每 lines_per_chunk 行切分为数据块。
    返回数据块列表（每个数据块为字符串）。
    """
    print(f"【Step 4】读取并切分文件 {file_path} ...")
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
    except Exception:
        # 如果文件不存在（模拟），则生成模拟内容
        content = "\n".join([f"Line {i+1} from {file_path}" for i in range(150)])
    lines = content.splitlines()
    chunks = ["\n".join(lines[i:i+lines_per_chunk]) for i in range(0, len(lines), lines_per_chunk)]
    print(f"文件 {file_path} 切分为 {len(chunks)} 个数据块")
    return chunks

def process_agent(agent_id: int, y1_file: str) -> None:
    """
    对单个 agent 任务进行问答处理：
      ① 从 y1_file 读取内容（或使用模拟数据）；
      ② 将内容按每 30 行切分为数据块；
      ③ 对每个数据块构造问答提示：首个数据块用 "语句A: ..."，后续用 "语句B: ..."；
      ④ 每次问答调用 qa_invoke，使用独立的对话记忆；
      ⑤ 将所有回答写入 answer_agent_{agent_id}.txt，同时在终端动态显示进度。
    """
    agent_name = f"agent_{agent_id}"
    print(f"[{agent_name}] 开始处理任务，源文件：{y1_file}")
    # 模拟读取 y1_file 内容（实际中请读取文件）
    try:
        with open(y1_file, "r", encoding="utf-8") as f:
            y1_content = f.read()
    except Exception:
        # 模拟内容
        y1_content = "\n".join([f"Processed line {i+1} from {y1_file}" for i in range(150)])
    chunks = split_into_chunks(y1_file)  # 或直接基于 y1_content 切分
    # 若需要根据 y1_content 切分，可用： chunks = [ ... ]（此处简化为调用 split_into_chunks 函数）
    num_chunks = len(chunks)
    conversation_memory = []  # 每个 agent 独立的对话记忆
    answer_file = f"answer_{agent_name}.txt"
    with open(answer_file, "w", encoding="utf-8") as f_out:
        for idx, chunk in enumerate(chunks):
            if idx == 0:
                prompt = f"语句A: {chunk}"
            else:
                prompt = f"语句B: {chunk}"
            # 将用户提问加入对话记忆
            conversation_memory.append({"role": "user", "content": prompt})
            # 调用 langgraph 问答接口
            response = qa_invoke(conversation_memory)
            # 假设 response 为对象，可取 response.content（否则直接用 str(response)）
            response_text = response.content if hasattr(response, "content") else str(response)
            conversation_memory.append({"role": "assistant", "content": response_text})
            # 写入答案文件
            f_out.write(response_text + "\n")
            print(f"[{agent_name}] 已处理数据块 {idx+1}/{num_chunks}")
            f_out.flush()
            time.sleep(0.1)  # 模拟处理延时
    print(f"[{agent_name}] 问答结果已保存至 {answer_file}")

def main():
    pcap_file = "input.pcap"
    # Step 1：分析 pcap 文件
    log_files = analyze_pcap(pcap_file)

    # Step 2：转换 log 文件为 txt 文件
    txt_files = convert_logs(log_files, n=10, m=5)

    # Step 3：筛选出有效的 txt 文件
    effective_txt = filter_effective_txt(txt_files)

    # Step 4：针对每个有效 txt 文件，进行字段提取生成 y1 文件（可能多个）
    agent_tasks = []
    for txt in effective_txt:
        y1_list = process_txt_file(txt)
        # 为每个 y1 文件生成一个任务，分配 agent_id（此处简单累加编号）
        for y1_file in y1_list:
            agent_tasks.append(y1_file)

    print(f"共计生成 {len(agent_tasks)} 个 agent 任务，准备并行处理...")
    # Step 5：并行处理每个 agent 任务
    with ThreadPoolExecutor(max_workers=len(agent_tasks)) as executor:
        futures = []
        for idx, y1_file in enumerate(agent_tasks, start=1):
            futures.append(executor.submit(process_agent, idx, y1_file))
        for future in as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Agent 任务处理异常：{e}")
    print("所有 agent 任务均已完成。")

if __name__ == "__main__":
    main()


【Step 1】分析 pcap 文件...
生成的 log 文件：['capture1.log', 'capture2.log', 'capture3.log']
【Step 2】转换 .log 文件为 .txt 文件...
转换得到的 txt 文件：['capture1_useful.txt', 'capture2_useless.txt', 'capture3_useful.txt']
筛选出有效 txt 文件：['capture1_useful.txt', 'capture3_useful.txt']
【Step 3】处理 capture1_useful.txt，提取字段生成 y1 文件...
生成的 y1 文件：['capture1_y1_1.txt', 'capture1_y1_2.txt']
【Step 3】处理 capture3_useful.txt，提取字段生成 y1 文件...
生成的 y1 文件：['capture3_y1.txt']
共计生成 3 个 agent 任务，准备并行处理...
[agent_1] 开始处理任务，源文件：capture1_y1_1.txt
[agent_2] 开始处理任务，源文件：capture1_y1_2.txt
[agent_3] 开始处理任务，源文件：capture3_y1.txt
【Step 4】读取并切分文件 capture1_y1_1.txt ...
【Step 4】读取并切分文件 capture1_y1_2.txt ...
文件 capture1_y1_1.txt 切分为 5 个数据块
【Step 4】读取并切分文件 capture3_y1.txt ...
文件 capture1_y1_2.txt 切分为 5 个数据块
文件 capture3_y1.txt 切分为 5 个数据块
Agent 任务处理异常：Checkpointer requires one or more of the following 'configurable' keys: ['thread_id', 'checkpoint_ns', 'checkpoint_id']
Agent 任务处理异常：Checkpointer requires one or more of the following 'configurable' keys: